<a href="https://colab.research.google.com/github/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# UC2 Pin Configurator

If you find an error, please report it [here](https://github.com/openUC2/UC2-REST/issues/new) 

In [1]:
!pip install UC2-REST

## Organize all imports

First of all we need to import the `ESP32Client`. Since it is not yet a standalone pip package, we have to do that via a relaitve import, meaning that the file is in the same folder as this Jupyter Notebook

In [11]:
%load_ext autoreload 
%autoreload 2
import uc2rest as uc2
import time 
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Connecting to the ESP32 via USB

Now we want to initiliaze the USB-serial connection. Therefore, connect the ESP32 to your computer using a data (!) USB cable and establish the connection. You can leave the port as "unknown" as the portfinder may identify the ESP.

**Important:** Close all applications that may be connected to the ESP (e.g. Arduino Serial Plotter)

**IMPORTANT:** Install the USB serial driver for the ESP32: https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all


**IMPORTANT**: If this fails => check the windows COM Ports which one is likely your port and rename "unknown" to e.g. `"COM3"`

In [12]:
# create the 
mPort = "unkown"
ESP32 = uc2.UC2Client(serialport=mPort)
ESP32.serial.DEBUG=False # only use if something is not working 


2022-11-15 01:13:05 DEBUG [UC2Client] /dev/cu.usbserial-0001
2022-11-15 01:13:08 DEBUG [UC2Client] We are connected: True on port: /dev/cu.usbserial-0001


# Load Pinconfiguration from the Device

Let's try to get the current pinconfiguration of the device. 

In [13]:
# see if it's the right device
ESP32.state.get_state()

# load the pin config from the ESP32
mConfig = ESP32.config.loadConfigDevice()
print(json.dumps(mConfig))


{"motXstp": 2, "motXdir": 33, "motYstp": 27, "motYdir": 16, "motZstp": 12, "motZdir": 14, "motAstp": 22, "motAdir": 21, "motEnable": 13, "ledArrPin": 4, "ledArrNum": 25, "digitalPin1": 0, "digitalPin2": 0, "analogPin1": 0, "analogPin2": 0, "analogPin3": 0, "laserPin1": 17, "laserPin2": 4, "laserPin3": 15, "dacFake1": 0, "dacFake2": 0, "identifier": "", "ssid": "", "PW": "", "PS3Mac": "", "PS4Mac": ""}


# Clear config and add your own 

The returning dictionary `mConfig` holds all the pin-pairs of the different external devices which we can change now. It will only be written to the device permanently in the next stage!

In [14]:
# reset the dictionary 
mConfig = dict.fromkeys(mConfig, 0)

# modify some values
mConfig["ledArrPin"] = 15
mConfig["ledArrNum"] = 16

mConfig["motXstp"] = 2
mConfig["motXdir"] = 33

mConfig["motYstp"] = 27
mConfig["motYdir"] = 16

mConfig["motZstp"] = 12
mConfig["motZdir"] = 14

mConfig["identifier"] = "UniEindhoven"
mConfig["ssid"] = "UC2"
mConfig["PW"] = "UC2"


# Write Pinconfiguration to the device

The `mConfig` will be written and readout after reboot of the microcontroller. 
Everything should be zero except the entries you modified. 

In [15]:
# see if it's the right device
# now load the config to the ESP32
ESP32.config.setConfigDevice(mConfig)
ESP32.config.applyConfigDevice()

# wait until the ESP reboots and identify the new config
time.sleep(5)

# see if it's the right device
ESP32.state.get_state()
mConfig = ESP32.config.loadConfigDevice()
print(mConfig)

{'motXstp': 2, 'motXdir': 33, 'motYstp': 27, 'motYdir': 16, 'motZstp': 12, 'motZdir': 14, 'motAstp': 0, 'motAdir': 0, 'motEnable': 0, 'ledArrPin': 15, 'ledArrNum': 16, 'digitalPin1': 0, 'digitalPin2': 0, 'analogPin1': 0, 'analogPin2': 0, 'analogPin3': 0, 'laserPin1': 0, 'laserPin2': 0, 'laserPin3': 0, 'dacFake1': 0, 'dacFake2': 0, 'identifier': '', 'ssid': '', 'PW': '', 'PS3Mac': '', 'PS4Mac': ''}


# Reset Pin Configuration to the default value 

The pin configuration will be reset to the values upon flashing it. 

In [16]:
ESP32.config.resetConfigDevice()
ESP32.config.applyConfigDevice()


In [17]:
# wait until the ESP reboots and identify the new config
time.sleep(5)

# see if it's the right device
ESP32.state.get_state()
mConfig = ESP32.config.loadConfigDevice()
print(mConfig)

{'motXstp': 2, 'motXdir': 33, 'motYstp': 27, 'motYdir': 16, 'motZstp': 12, 'motZdir': 14, 'motAstp': 22, 'motAdir': 21, 'motEnable': 13, 'ledArrPin': 32, 'ledArrNum': 25, 'digitalPin1': 0, 'digitalPin2': 0, 'analogPin1': 0, 'analogPin2': 0, 'analogPin3': 0, 'laserPin1': 17, 'laserPin2': 4, 'laserPin3': 15, 'dacFake1': 0, 'dacFake2': 0, 'identifier': '', 'ssid': '', 'PW': '', 'PS3Mac': '', 'PS4Mac': ''}
